<a href="https://colab.research.google.com/github/CollinScott37/CollinScott37/blob/main/ACG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Content:
##Formatting Data
##Models
##Results

_______________________________________________________________________

#Formatting Data:

In [ ]:
#box 1

#upload data file via file "icon" on left (not top left word "file")

import numpy as np
import pandas as pd
import time
import datetime

data_frame = pd.read_csv('ks-projects-201801.csv')
data_frame.head()


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 4:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 0:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 3:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 8:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [ ]:
#box 2

#**********************************************
#Removes colums and rows not needed in the data
#**********************************************

from re import L
labels = data_frame.values[:, 9]
features = data_frame.values[:, :]
#print(features[0])

#Clean Up
#OLD COLUMN ORDER: category, main_category, deadline, launched, usd_pledged_real, usd_gaol_real
features = np.delete(features, [0,1,4,6,8,9,11,12], 1) #remove certain rows

#swaps deadline and launch so launch is listed first
def Swap(arr, start_index, last_index):
    arr[:, [start_index, last_index]] = arr[:, [last_index, start_index]]

#NEW COLUMN ORDER: category, main_category, launched, deadline, usd_pledged_real, usd_gaol_real
Swap(features, 2, 3)


#the states we don't want
unwantedLabels = np.array(['canceled', 'live', 'suspended', 'undefined'])

#row index of the with states we don't want
rowIndexToDelete = np.zeros(labels.shape)
for i  in range(len(unwantedLabels)):
  rowIndexToDelete += labels == unwantedLabels[i]

#convert to bool array
rowIndexToDelete = rowIndexToDelete.astype(bool)


#delete those rows
labels = np.delete(labels, rowIndexToDelete, axis=0)
features = np.delete(features, rowIndexToDelete, axis=0)

print('array of labels: shape ' + str(np.shape(labels)))
print('array of feature matrix: shape ' + str(np.shape(features)))


array of labels: shape (331675,)
array of feature matrix: shape (331675, 7)


In [ ]:
#box 3

#***********************************
#Formating data to read time legibly
#***********************************

#COLUMN ORDER: category, main_category, launched, deadline, backers, usd_pledged_real, usd_gaol_real

#convert launched(string) to time in seconds
for i in range(len(features)):
  features[i][2] = datetime.datetime.strptime(features[i][2], "%Y-%m-%d %H:%M:%S").timestamp()

#convert deadline(string) to time in seconds
for i in range(len(features)):
  features[i][3] = datetime.datetime.strptime(features[i][3], "%Y-%m-%d").timestamp()

#print(features[0][2])
#print(features[0][3])

In [ ]:
#box 4

#******************************************************
#Hot One Encoding for category and Main category columns
#******************************************************


#COLUMN ORDER: sub_category, main_category, launched, deadline, backers, usd_pledged_real, usd_gaol_real

MAIN_CAT_SIZE =  np.unique(features[:,1]).size
SUB_CAT_SIZE = np.unique(features[:,0]).size

#new shape should be (331675, 179)
#Hot One Encoding for both sub_cat and main_cat FOR FEATURES
nf = np.zeros((len(features), 179))
subCatUnique = np.unique(features[:,0])
mainCatUnique = np.unique(features[:,1])
for i in range(len(features)):
  subMatch = (subCatUnique == features[i][0]).astype(int)
  mainMatch = (mainCatUnique == features[i][1]).astype(int)
  x = np.insert(features[i],1,subMatch)
  y = np.insert(x,-5, mainMatch)
  y = y[1:]
  nf[i] = np.append(y[:SUB_CAT_SIZE],y[SUB_CAT_SIZE+1:])

 

In [ ]:
#box 5

#***************************
#Hot one encoding for labels
#***************************

uniqueLabels = np.unique(labels)
print(uniqueLabels)

mask = labels == uniqueLabels[1]

labels = mask.astype(int)


#0 == failed
#1 == successful

['failed' 'successful']


In [ ]:
#box 6

#*********************************
#Maintains an old copy of features
#*********************************

oldFeatures = features;
features = nf

In [ ]:
#box 7

#***********************************************************
#Generates %80 training data set and %20 validation data set
#***********************************************************

np.random.seed(seed=1)
def IDK (features, labels):
  #20% of the data
  validAmount = len(features) * 0.2

  #list of all possible indicies inisde features
  indices = np.arange(0, len(features))

  #random choose indicies (20% of features)
  validIndicies = np.random.choice(indices, size=int(validAmount), replace=False)
  validIndicies = np.sort(validIndicies)

  #list of all possible indicies inisde features
  indices = np.arange(0, len(features))

  #random choose indicies (20% of features)
  validIndicies = np.random.choice(indices, size=int(validAmount), replace=False)

  #The Validation Set
  validSet = np.empty((int(validAmount), len(features[0])))
  validLabels = np.empty((int(validAmount)))

  #add the respective indicies for the validation set
  for i in range(len(validSet)):
      validSet[i] = np.copy(features[validIndicies[i]])
      validLabels[i] = np.copy(labels[validIndicies[i]])

  #remove those indicies from features
  trainingSet = np.delete(features, validIndicies, axis=0)
  trainingLabels = np.delete(labels, validIndicies);
  return (validSet, validLabels, trainingSet, trainingLabels)

validSet, validLabels, trainingSet, trainingLabels = IDK(features, labels)

print("validSet:")
print(validSet.shape)
print("validLabels:")
print(validLabels.shape)
print("trainingSet:")
print(trainingSet.shape)
print("trainingLabels:")
print(trainingLabels.shape);

print("features:")
print(features.shape)
print("labels:")
print(labels.shape);


validSet:
(66335, 179)
validLabels:
(66335,)
trainingSet:
(265340, 179)
trainingLabels:
(265340,)
features:
(331675, 179)
labels:
(331675,)


In [ ]:
#7.5 testing
test = np.delete(features, [-5,-4], 1) #remove launched and dealine

TvalidSet, TvalidLabels, TtrainingSet, TtrainingLabels = IDK(test, labels)

In [ ]:
#box 8
#print(TtrainingSet)
#print(TvalidSet)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10, weights='distance', p=2)
knn.fit(TtrainingSet, TtrainingLabels)
print(f"{}", knn.predict(TvalidSet[0]))
knn.score(TvalidSet[0:100], TvalidLabels[0:100])

[[0.00000e+00 0.00000e+00 0.00000e+00 ... 0.00000e+00 0.00000e+00
  1.53395e+03]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 3.00000e+00 2.20000e+02
  4.50000e+04]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 1.00000e+00 1.00000e+00
  5.00000e+03]
 ...
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 5.00000e+00 1.55000e+02
  1.50000e+03]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 1.00000e+00 2.00000e+01
  1.50000e+04]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 1.70000e+01 5.24000e+02
  2.00000e+03]]
[[0.000000e+00 0.000000e+00 0.000000e+00 ... 5.000000e+00 1.333300e+02
  1.500000e+03]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 2.190000e+02 1.733740e+04
  1.250782e+04]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 1.020000e+02 1.326500e+04
  4.500000e+04]
 ...
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 1.400000e+01 8.800000e+02
  8.000000e+02]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 2.200000e+01 1.550000e+03
  9.000000e+02]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 5.700000e+01 2.969000e+

1.0

# Models
##insert all models here

###Feauture Selection:
###PCA:


#Results